**1. Annotated Corpus for Named Entity Recognition: Corpus for entity classification with enhanced and popular features by Natural Language Processing applied to the data set.**

- https://www.kaggle.com/abhinavwalia95/entity-annotated-corpus


Essential info about entities:

- geo = Geographical Entity
- org = Organization
- per = Person
- gpe = Geopolitical Entity
- tim = Time indicator
- art = Artifact
- eve = Event
- nat = Natural Phenomenon

In [3]:
import os 
import pandas as pd

In [8]:
dframe = pd.read_csv("./data/Annotated/ner.csv", encoding = "ISO-8859-1", error_bad_lines=False)
dframe.head()

b'Skipping line 281837: expected 25 fields, saw 34\n'


,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [9]:
dataset=dframe.drop(['Unnamed: 0', 'lemma', 'next-lemma', 'next-next-lemma', 'next-next-pos',
       'next-next-shape', 'next-next-word', 'next-pos', 'next-shape',
       'next-word', 'prev-iob', 'prev-lemma', 'prev-pos',
       'prev-prev-iob', 'prev-prev-lemma', 'prev-prev-pos', 'prev-prev-shape',
       'prev-prev-word', 'prev-shape', 'prev-word','shape'],axis=1)

In [10]:
dataset.head()

,pos,sentence_idx,word,tag
0,NNS,1.0,Thousands,O
1,IN,1.0,of,O
2,NNS,1.0,demonstrators,O
3,VBP,1.0,have,O
4,VBN,1.0,marched,O


In [22]:
dataset.tag.unique()

array(['O', 'B-geo', 'B-gpe', 'B-per', 'I-geo', 'B-org', 'I-org', 'B-tim',
       'B-art', 'I-art', 'I-per', 'I-gpe', 'I-tim', 'B-nat', 'B-eve',
       'I-eve', 'I-nat', nan], dtype=object)

**2. CoNLL 2003: Dataset that contains 1,393 English news articles with annotated entities LOC (location), ORG (organization), PER (person) and MISC (miscellaneous).**

- https://www.clips.uantwerpen.be/conll2003/ner/

----
something like this

 | TOKENS | POS | CHUNK_TAG| NE|
 | --- |   ---  | --- | -- |   
 |U.N.        | NNP | I-NP | I-ORG |
 | official   | NN  | I-NP | O |
 | Ekeus   |     NNP | I-NP | I-PER |
 | heads  |      VBZ | I-VP | O |
 | for       |   IN |  I-PP | O| 
 | Baghdad  |    NNP | I-NP | I-LOC |
 |  .    |        .   | O  |   O |
 
 
 - Tokens : word in sentence
 - POS: part of speech(词性)
 - CHUNK_TAG: syntactic chunk tag(句法标记块)
 - NE：label
 
 
 > from https://www.clips.uantwerpen.be/conll2003/ner/    The CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on a separate line and there is an empty line after each sentence. The first item on each line is a word, the second a part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags and the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only if two phrases of the same type immediately follow each other, the first word of the second phrase will have tag B-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase.
 

In [13]:
for dirname, _, filenames in os.walk('./data/CONLL003'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./data/CONLL003\test.txt
./data/CONLL003\train.txt
./data/CONLL003\valid.txt


In [14]:
def load_sentences(filepath):
    """
    Load sentences (separated by newlines) from dataset

    Parameters
    ----------
    filepath : str
        path to corpus file

    Returns
    -------
    List of sentences represented as dictionaries

    """
    
    sentences, tok, pos, chunk, ne = [], [], [], [], []

    with open(filepath, 'r') as f:
        for line in f.readlines():
            if line == ('-DOCSTART- -X- -X- O\n') or line == '\n':
               # Sentence as a sequence of tokens, POS, chunk and NE tags
                sentence = dict({'TOKENS' : [], 'POS' : [], 'CHUNK_TAG' : [], 'NE' : [], 'SEQ' : []})
                sentence['TOKENS'] = tok
                sentence['POS'] = pos
                sentence['CHUNK_TAG'] = chunk
                sentence['NE'] = ne
               
                # Once a sentence is processed append it to the list of sentences
                sentences.append(sentence)
               
                # Reset sentence information
                tok = []
                pos= []
                chunk = []
                ne = []
            else:
                l = line.split(' ')
               
                # Append info for next word
                tok.append(l[0])
                pos.append(l[1])
                chunk.append(l[2])
                ne.append(l[3].strip('\n'))
    
    return sentences

In [15]:
train = load_sentences('./data/CONLL003/train.txt')

In [19]:
len(train)

15933

In [21]:
# example
train[3]

{'TOKENS': ['Peter', 'Blackburn'],
 'POS': ['NNP', 'NNP'],
 'CHUNK_TAG': ['B-NP', 'I-NP'],
 'NE': ['B-PER', 'I-PER'],
 'SEQ': []}

In [1]:
l = [x.strip() for x in open('../data/CONLL003/class.txt').readlines()]
l

['I-LOC', 'B-ORG', 'O', 'B-PER', 'I-PER', 'I-MISC', 'B-MISC', 'I-ORG', 'B-LOC']

- Reference from https://lionbridge.ai/datasets/15-free-datasets-and-corpora-for-named-entity-recognition-ner/ and other sources
- Only for group project 
    - group member: 陈， 古， 黄， 罗